In [6]:
from abc import abstractmethod, ABC
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
df = pd.read_pickle("../data/master_data.pkl")
# tfidf = Recommender(data=df)

# query = "Attention mechanism, gpt"
# res = tfidf.recommend(query)

# print(res.shape)
# print(res)
print(df.shape)
df.head(20)

(100, 7)


,title,published_date,pdf_link,summary,pdf_text,summary_length,pdf_text_length
http://arxiv.org/abs/2401.09420v1,LionHeart: A Layer-based Mapping Framework for...,2024-01-17 18:59:26,http://arxiv.org/pdf/2401.09420v1,"When arranged in a crossbar configuration, res...",LionHeart: A Layer-based Mapping Framework for...,1027,31764
http://arxiv.org/abs/2401.09419v1,GARField: Group Anything with Radiance Fields,2024-01-17 18:57:53,http://arxiv.org/pdf/2401.09419v1,Grouping is inherently ambiguous due to the mu...,GARField: Group Anything with Radiance Fields\...,1366,55047
http://arxiv.org/abs/2401.09417v1,Vision Mamba: Efficient Visual Representation ...,2024-01-17 18:56:18,http://arxiv.org/pdf/2401.09417v1,Recently the state space models (SSMs) with ef...,Vision Mamba: Efficient Visual Representation ...,1491,43678
http://arxiv.org/abs/2401.09416v1,TextureDreamer: Image-guided Texture Synthesis...,2024-01-17 18:55:49,http://arxiv.org/pdf/2401.09416v1,"We present TextureDreamer, a novel image-guide...",TextureDreamer: Image-guided Texture Synthesis...,1394,50400
http://arxiv.org/abs/2401.09415v1,Randomized Kaczmarz with geometrically smoothe...,2024-01-17 18:55:24,http://arxiv.org/pdf/2401.09415v1,This paper studies the effect of adding geomet...,RANDOMIZED KACZMARZ WITH GEOMETRICALLY\nSMOOTH...,434,43597
http://arxiv.org/abs/2401.09414v1,Vlogger: Make Your Dream A Vlog,2024-01-17 18:55:12,http://arxiv.org/pdf/2401.09414v1,"In this work, we present Vlogger, a generic AI...",Vlogger: Make Your Dream A Vlog\nShaobin Zhuan...,1609,76883
http://arxiv.org/abs/2401.09413v1,POP-3D: Open-Vocabulary 3D Occupancy Predictio...,2024-01-17 18:51:53,http://arxiv.org/pdf/2401.09413v1,We describe an approach to predict open-vocabu...,POP-3D: Open-Vocabulary 3D Occupancy Predictio...,1406,63715
http://arxiv.org/abs/2401.09412v1,Weakly-Private Information Retrieval From MDS-...,2024-01-17 18:51:04,http://arxiv.org/pdf/2401.09412v1,We consider the problem of weakly-private info...,arXiv:2401.09412v1 [cs.IT] 17 Jan 2024\nWeak...,521,12593
http://arxiv.org/abs/2401.09410v1,Through the Looking-Glass: Transparency Implic...,2024-01-17 18:47:30,http://arxiv.org/pdf/2401.09410v1,Knowledge can't be disentangled from people. A...,Through the Looking-Glass: Transparency Implic...,1442,90887
http://arxiv.org/abs/2401.09407v1,Deciphering Textual Authenticity: A Generalize...,2024-01-17 18:45:13,http://arxiv.org/pdf/2401.09407v1,With the recent proliferation of Large Languag...,Deciphering Textual Authenticity: A Generalize...,1801,78309


In [12]:
def preprocess(text):
        """word tokenization, snowball stemming and punctuation removal"""
        stemmer=SnowballStemmer("english")
        return [stemmer.stem(token) for token in word_tokenize(text) if token.isalpha()]

print(preprocess(df.pdf_text.values[0][:100]))

['lionheart', 'a', 'map', 'framework', 'for', 'heterogen', 'system', 'with', 'analog', 'comput']


In [14]:
def learn_vocabulary(corpus): 
    english_stopwords=stopwords.words("english")
    vectorizer=TfidfVectorizer(
        lowercase=True,
        tokenizer=preprocess,
        stop_words=english_stopwords,
        ngram_range=(1,2)
    )
    vocabulary=vectorizer.fit(corpus)
    return (vectorizer, vocabulary)
    
vectorizer, vocabulary=learn_vocabulary(df.pdf_text)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'might', 'must', 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'veri', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  warnings.warn(


In [15]:
vectorizer

TfidfVectorizer(ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function preprocess at 0x1304ff1a0>)

In [16]:
vocabulary

TfidfVectorizer(ngram_range=(1, 2),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function preprocess at 0x1304ff1a0>)

In [21]:
len(vocabulary.vocabulary_)

339873

In [20]:
vocabulary.get_feature_names_out()

array(['aa', 'aa grate', 'aa let', ..., '𝜙𝑙𝑟 crm', '𝜙𝑚𝑟', '𝜙𝑚𝑟 𝜙𝑙𝑟'],
      dtype=object)

In [17]:
def transform_data(data):
    transformed_data=vectorizer.transform(data)
    return transformed_data
    
transformed_data=transform_data(df.pdf_text)

In [23]:
transformed_data.shape

(100, 339873)

In [24]:
def top10(arr, results=10):
        kth_largest = (results + 1) * -1
        return np.argsort(arr)[:kth_largest:-1]

In [45]:
np.argsort([45,3,2,99,8,23,42, 78,68,5,4,55])

array([ 2,  1, 10,  9,  4,  5,  6,  0, 11,  8,  7,  3])

In [51]:
np.argsort([45,3,2,99,8,23,42, 78,68,5,4,55])[:-10:-1]

array([ 3,  7,  8, 11,  0,  6,  5,  4,  9])

In [138]:
transformed_query=vectorizer.transform(["Attention mechanism, gpt"]) 
transformed_query.shape

(1, 339873)

In [139]:
cosine_similarities = cosine_similarity(
        X=transformed_query,
        Y=transformed_data
    )
cosine_similarities.shape

(1, 100)

In [142]:
"Attention" in df.pdf_text.values[40]

True

In [141]:
sorted(list(enumerate(cosine_similarities[0])), reverse=True, key=lambda x:x[1])

[(40, 0.07523317245996379),
 (30, 0.0645704279199833),
 (68, 0.019730676950393673),
 (66, 0.01706847010186102),
 (88, 0.01676402091510068),
 (9, 0.015864706383126106),
 (5, 0.012539847098876067),
 (2, 0.011670485300686375),
 (84, 0.009874931918079605),
 (51, 0.009075828413454008),
 (73, 0.007463179286442995),
 (42, 0.007174499216149932),
 (76, 0.00707560270301927),
 (24, 0.005183716246368177),
 (85, 0.005024141411999368),
 (50, 0.004790685676727951),
 (81, 0.004607006211813749),
 (99, 0.004537051727032916),
 (8, 0.004231346499790698),
 (79, 0.003719270429699103),
 (28, 0.0036587401915262013),
 (67, 0.0030915648551165464),
 (77, 0.0029429730565885947),
 (35, 0.0027201906268400932),
 (80, 0.002680087917726067),
 (31, 0.002592577001522031),
 (87, 0.0025478606758592495),
 (94, 0.0024699564996087725),
 (34, 0.0024318648369029945),
 (95, 0.0024127870322119063),
 (17, 0.0022197657055622894),
 (41, 0.0020517473523855496),
 (63, 0.0019436895817855633),
 (47, 0.0016778977356507368),
 (58, 0.0016

In [118]:
np.argmax(cosine_similarities[0])

34

In [123]:
df.pdf_text.values[34]

'Large Language Models Are Neurosymbolic Reasoners\nMeng Fang*1,2, Shilong Deng*1, Yudi Zhang*2,\nZijing Shi3, Ling Chen3, Mykola Pechenizkiy2, Jun Wang4\n1University of Liverpool, United Kingdom\n2Eindhoven University of Technology, Netherlands\n3University of Technology Sydney, Australia\n4University College London, United Kingdom\n{Meng.Fang, shilong.deng}@liverpool.ac.uk, {y.zhang5, m.pechenizkiy}@tue.nl,\nzijing.shi@student.uts.edu.au, ling.chen@uts.edu.au, j.wang@cs.ucl.ac.uk\nAbstract\nA wide range of real-world applications is characterized by\ntheir symbolic nature, necessitating a strong capability for\nsymbolic reasoning. This paper investigates the potential ap-\nplication of Large Language Models (LLMs) as symbolic\nreasoners. We focus on text-based games, significant bench-\nmarks for agents with natural language capabilities, particu-\nlarly in symbolic tasks like math, map reading, sorting, and\napplying common sense in text-based worlds. To facilitate\nthese agents, we

In [89]:
cosine_similarities

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.07752967, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.17667167, 0.01348768],
       [0.01564975, 0.        ],
       [0.        , 0.        ],
       [0.0390098 , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.

In [82]:
cosine_similarity(transformed_data).shape

(100, 100)

In [79]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(transformed_query,transformed_data).flatten()
cosine_similarities

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.92680207e-03,
       0.00000000e+00, 0.00000000e+00, 2.49027927e-02, 2.49027927e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.96903720e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 6.68027204e-03, 3.59684430e-03, 0.00000000e+00,
       3.06247952e-02, 3.06247952e-02, 0.00000000e+00, 0.00000000e+00,
       1.84663789e-03, 1.19313745e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.22396398e-03, 0.00000000e+00, 3.80478098e-03, 3.80478098e-03,
       0.00000000e+00, 0.00000000e+00, 1.23888779e-02, 5.00288737e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [78]:
related_docs_indices = cosine_similarities.argsort()[:-6:-1]
related_docs_indices

array([319, 578, 495, 627, 194])

In [33]:
top10(cosine_similarities, results=10)

array([[0, 1, 2, 3, 4, 5, 6],
       [0, 1, 2, 3, 6, 5, 4],
       [0, 1, 2, 3, 6, 4, 5],
       [2, 3, 4, 6, 0, 1, 5],
       [0, 1, 2, 3, 4, 5, 6],
       [0, 1, 2, 3, 4, 5, 6],
       [2, 3, 6, 0, 1, 4, 5],
       [0, 1, 2, 3, 4, 5, 6],
       [2, 3, 6, 0, 1, 5, 4],
       [0, 1, 2, 3, 4, 5, 6]])

In [55]:
np.argsort(cosine_similarities, axis=0)[::-1][:10]

array([[72, 72, 99, 99, 45, 70, 99],
       [60, 60, 36, 36, 82, 27, 36],
       [22, 22, 26, 26, 89, 97, 26],
       [28, 28, 27, 27, 60, 49, 27],
       [44, 44, 28, 28, 91, 57, 28],
       [47, 47, 29, 29, 42, 25, 29],
       [70, 70, 30, 30, 97, 45, 30],
       [27, 27, 31, 31, 59, 32, 31],
       [84, 84, 32, 32, 43, 59, 32],
       [ 4,  4, 33, 33, 22, 64, 33]])

In [56]:
indices = np.argsort(cosine_similarities, axis=0)[::-1]
top_10_indices = indices[:10, :]
top_10_indices

array([[72, 72, 99, 99, 45, 70, 99],
       [60, 60, 36, 36, 82, 27, 36],
       [22, 22, 26, 26, 89, 97, 26],
       [28, 28, 27, 27, 60, 49, 27],
       [44, 44, 28, 28, 91, 57, 28],
       [47, 47, 29, 29, 42, 25, 29],
       [70, 70, 30, 30, 97, 45, 30],
       [27, 27, 31, 31, 59, 32, 31],
       [84, 84, 32, 32, 43, 59, 32],
       [ 4,  4, 33, 33, 22, 64, 33]])

In [62]:
top_10_indices[0,0]

72

In [58]:
np.take_along_axis(cosine_similarities, top_10_indices, axis=0)[0]

array([0.05886475, 0.05886475, 0.        , 0.        , 0.24116647,
       0.13024936, 0.        ])

In [61]:
# Get the indices of the most similar documents
indices = np.argsort(cosine_similarities, axis=0)[::-1]

# Get the top 10 indices and similarity values
top_10_indices = indices[:10, :]
top_10_similarities = np.take_along_axis(cosine_similarities, top_10_indices, axis=0)

query1="llm attention"
query2="gpt chat large"
n_query=[query1, query2]
# Print the results
for i in range(len(n_query)):
    print(f"Query {i}:")
    for j in range(10):
        print(f"Document {top_10_indices[j, i]}: {top_10_similarities[j, i]:.4f}")
    print()

Query 0:
Document 72: 0.0589
Document 60: 0.0547
Document 22: 0.0541
Document 28: 0.0526
Document 44: 0.0482
Document 47: 0.0427
Document 70: 0.0414
Document 27: 0.0376
Document 84: 0.0351
Document 4: 0.0306

Query 1:
Document 72: 0.0589
Document 60: 0.0547
Document 22: 0.0541
Document 28: 0.0526
Document 44: 0.0482
Document 47: 0.0427
Document 70: 0.0414
Document 27: 0.0376
Document 84: 0.0351
Document 4: 0.0306



In [ ]:
def recommend(query:str):
    transformed_query=vectorizer.transform(list(query)) 
    cosine_similarities = cosine_similarity(
        X=transformed_data,
        Y=transformed_query
    ).flatten()
    top_related_indices = top10(cosine_similarities)
    return top_related_indices

In [ ]:
from abc import abstractmethod, ABC
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download("stopwords")

class BaseRecommender(ABC):
    @abstractmethod
    def preprocess(self):
        pass

    @abstractmethod
    def learn_vocabulary(self):
        pass

    @abstractmethod
    def transform_data(self):
        pass

    @abstractmethod
    def top10(self):
        pass

    @abstractmethod
    def recommend(self):
        pass


class Recommender(BaseRecommender):
    def __init__(self, data:pd.DataFrame) -> None:
        self.data=data
        self.corpus=self.data["pdf_text"]
        self.learn_vocabulary()
        self.transform_data()
        super().__init__()
    
    # tokenize, stem and remove punctuation
    def preprocess(self,text):
        """word tokenization, snowball stemming and punctuation removal"""
        stemmer=SnowballStemmer("english")
        return [stemmer.stem(token) for token in word_tokenize(text) if token.isalpha()]
    
    def learn_vocabulary(self): 
        english_stopwords=stopwords.words("english")
        self.vectorizer=TfidfVectorizer(
            lowercase=True,
            tokenizer=self.preprocess,
            stop_words=english_stopwords,
            ngram_range=(1,2)
        )
        self.vocabulary=self.vectorizer.fit(self.corpus)

    def transform_data(self):
        self.transformed_data=self.vectorizer.transform(self.corpus)
    
    def top10(self, arr, results=10):
        kth_largest = (results + 1) * -1
        return np.argsort(arr)[:kth_largest:-1]

    def recommend(self, query:str):
        transformed_query=self.vectorizer.transform(list(query)) 
        cosine_similarities = cosine_similarity(
            X=self.transformed_data,
            Y=transformed_query
        ).flatten()
        top_related_indices = self.top10(cosine_similarities)
        return top_related_indices
    
if __name__ == "__main__":
    df = pd.read_pickle("../data/master_data.pkl")
    tfidf = Recommender(data=df)

    query = "Attention mechanism, gpt"
    res = tfidf.recommend(query)

    print(res.shape)
    print(res)